In [ ]:
import json
import re

from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
line_bot_api = LineBotApi(env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    emojis = event.message.emojis  # this variable exists when user sends emojis
    text = event.message.text
    if emojis:
        text = re.sub('\([a-z 0-9]+\)', '$', text)  # pos of emojis in text must be marked as $
        text_is = [m.start() for m in re.finditer('[$]{1}', text)]
        for i in range(len(emojis)):  # emoji.index must be the pos of emoji in text
            emojis[i].index = text_is[i]
    
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=text, emojis=emojis))


if __name__ == "__main__":
    app.run()